In [2]:
import pandas as pd
from itertools import product
from amplify import BinaryPoly, gen_symbols, sum_poly, Solver, decode_solution
from amplify.constraint import equal_to, greater_equal
from amplify.client import FixstarsClient
import numpy as np
from collections import defaultdict

# 各店舗の要求役割情報
dict_req = dict(
    location=["tenjin", "hakata", "akasaka", "gakken"],
    manager=[1, 1, 1, 1],
    submanager=[1, 0, 1, 1],
    employee=[2, 2, 2, 2],
)
df_req = pd.DataFrame.from_dict(dict_req, orient="index").T

# 各従業員の勤務希望情報
dict_worker_loc = dict(
    worker_id=[0, 1, 2, 3, 4, 5, 6, 7, 8],
    tenjin=[2, 0, 0, 0, 1, 1, 2, 1, 1],
    hakata=[1, 0, 0, 2, 2, 2, 1, 2, 1],
    akasaka=[1, 0, 0, 1, 0, 1, 1, 1, 2],
    gakken=[1, 2, 2, 0, 0, 0, 0, 0, 0],
)
df_worker_loc = pd.DataFrame.from_dict(dict_worker_loc, orient="index").T

# 各従業員のスキル情報
dict_worker_skill = dict(
    worker_id=[0,1,2,3,4,5,6,7,8],
    manager=[1,1,0,0,1,1,1,0,1],
    submanager=[1,1,1,0,1,1,1,0,1],
    employee=[1,1,1,1,1,1,1,1],
)
df_worker_skill = pd.DataFrame.from_dict(dict_worker_skill, orient="index").T

df_req = pd.DataFrame.from_dict(dict_req, orient="index").T
print("各店舗の従業員要求人数")
display(df_req.style.hide_index())
df_worker_loc = pd.DataFrame.from_dict(dict_worker_loc, orient="index").T
print("\n各従業員の勤務希望情報")
display(df_worker_loc.style.hide_index())

# 従業員id
workers = df_worker_loc["worker_id"].values
# 店舗名
locations = df_req["location"].values
# 役職名
roles = ["manager", "submanager", "employee"]

## 各データ長を取得
num_workers = len(workers)
num_locations = len(locations)
num_roles = len(roles)

# 店舗名<->店舗番号の変換をするdictを作成
idx2loc = dict((i, v) for i, v in enumerate(df_req["location"].values))
loc2idx = dict((v, i) for i, v in enumerate(df_req["location"].values))

# 役職名<->役職番号の変換をするdictを作成
idx2role = dict((i,v) for i, v in enumerate(roles))
role2idx = dict((v,i) for i, v in enumerate(roles))

# 決定変数を用意
location_variables = gen_symbols(BinaryPoly, num_workers, num_locations)
# 従業員iが役職jで店舗lに勤務すること表す変数
role_variables = gen_symbols(BinaryPoly, num_workers, num_roles, num_locations)

# 勤務不可能店舗に関しては変数を０へ
for i, l in product(range(num_workers), locations):
    worker_req = df_worker_loc.iloc[i][l]
    if worker_req == 0:
        # すべての役職で店舗割当が不可
        for j in range(num_roles):
            role_variables[i][j][loc2idx[l]] = BinaryPoly(0)
for i, j in product(range(num_workers), roles):
    worker_skill = df_worker_skill.iloc[i][j]
    if worker_skill == 0:
        # すべての店舗で役職が不可
        for l in range(num_locations):
            role_variables[i][role2idx[j]][l] = BinaryPoly(0)

# M->L
location_variables = [
    [
        sum_poly(num_roles, lambda j: role_variables[i][j][l])
        for l in range(num_locations)
    ]
    for i in range(num_workers)
]

# 充足率の計算
w = [
    (sum_poly(num_workers, lambda i: location_variables[i][l])) / df_req["employee"][l]
    for l in range(num_locations)
]

# 充足率の平均の-1倍
average_fill_rate_cost = -((sum_poly(w) / len(w)) )

# 充足率の分散
variance_fill_rate_cost = (
    sum_poly(len(w), lambda i: w[i] ** 2) / len(w) - (sum_poly(w) / num_locations) ** 2
)

# 従業員の希望度の-1倍
location_cost = -sum_poly(
    num_workers,
    lambda i: sum_poly(
        num_locations,
        lambda l: df_worker_loc.loc[i][idx2loc[l]] * location_variables[i][l],
    ),
)

# 従業員iは同時に1店舗のみ勤務できる
location_constarints = sum(
    [equal_to(sum_poly(location_variables[i]), 1) for i in range(num_workers)]
)

# 各店舗の要求人数に等しい店長、副店長を配置する
req_manager_constraints = sum(
    [
        equal_to(
            sum_poly(num_workers, lambda i: role_variables[i][0][l]),
            df_req["manager"][l],
        )
        for l in range(num_locations)
    ]
)

req_submanager_constraints = sum(
    [
        equal_to(
            sum_poly(num_workers, lambda i: role_variables[i][1][l]),
            df_req["submanager"][l],
        )
        for l in range(num_locations)
    ]
)

# 店舗の合計人数は要求人数以上
req_employee_constraints = sum(
    [
        greater_equal(
            sum_poly(num_workers, lambda i: location_variables[i][l]),
            df_req["employee"][l],
        )
        for l in range(num_locations)
    ]
)

# それぞれの目的関数の係数
loc_priority = 1
ave_fill_priority = 1
var_fill_priority = 10

# 目的関数
cost_func = (
    loc_priority * location_cost
    + ave_fill_priority * average_fill_rate_cost
    + var_fill_priority * variance_fill_rate_cost
)

# 制約条件を表すペナルティ関数の重み
constraint_weight = 20

# 制約条件
constraints = constraint_weight * (
    location_constarints 
    + req_manager_constraints
    + req_submanager_constraints
    + req_employee_constraints
)

# 最適化モデル
model = cost_func + constraints

# クライアントの設定
client = FixstarsClient()
client.token = "UgSe16jyscbTusQWa4jRhtUn8rVypQgc"
client.parameters.timeout = 1000  #  タイムアウト1秒

# ソルバーを定義して実行
solver = Solver(client)
result = solver.solve(model)

# 制約条件チェック
if len(result) == 0:
    raise RuntimeError("The given constraints are not satisfied")
values = result[0].values
energy = result[0].energy

# 勤務地に関する変数の解
location_solutions = decode_solution(location_variables, values, 0)

# 割当店舗と役職に関する変数の解
role_solutions = decode_solution(role_variables, values)

# 結果を格納する変数を生成
(role_index_list, loc_index_list) = np.where(np.array(role_solutions) == 1)[1:]
dict_df = defaultdict(list)

for i, (j,l) in enumerate(zip(role_index_list, loc_index_list)):
    ## 配属勤務地
    worker_id = df_worker_loc.loc[i]["worker_id"]
    role = roles[j]
    loc = locations[l]
    dict_df["worker_id"].append(worker_id)
    dict_df["role"].append(role)
    dict_df["location"].append(loc)

df_result = pd.DataFrame.from_dict(dict_df, orient="index").T
display(df_result.style.hide_index())

各店舗の従業員要求人数


location,manager,submanager,employee
tenjin,1,1,2
hakata,1,0,2
akasaka,1,1,2
gakken,1,1,2



各従業員の勤務希望情報


worker_id,tenjin,hakata,akasaka,gakken
0,2,1,1,1
1,0,0,0,2
2,0,0,0,2
3,0,2,1,0
4,1,2,0,0
5,1,2,1,0
6,2,1,1,0
7,1,2,1,0
8,1,1,2,0


worker_id,role,location
0,manager,tenjin
1,manager,gakken
2,submanager,gakken
3,employee,hakata
4,manager,hakata
5,submanager,akasaka
6,submanager,tenjin
7,employee,hakata
8,manager,akasaka
